<a href="https://colab.research.google.com/github/Rossel/Digital_Music_Store/blob/master/Chinook_Music_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chinook Music Store**

## 1. Introduction

In this report we will be analzing a database for a *fictional* digital music shop, comparable to the iTunes Music store, called Chinook. The fictional management wants insights in its sales performance to further optimize its business strategy. 
The central question being asked by Chinook's management is:

**How can Chinook improve its performance and optimize its sales strategy?**


In order to answer this question, and various sub questions, this report will cover the following topics:

* Best selling product categories
* Sales performance per country
* Sales agents performance
* Customer base
* Music inventory

![Chinook logo](https://rlv.zcache.com/chinook_salmon_oval_logo_oval_sticker-rc0dcc9a4f9ef4155b430ba4da2d23edd_0ugdd_8byvr_630.jpg)

###The database

For this report a modified version of a SQLite database is being used from [this resource](https://github.com/lerocha/chinook-database). The actual version used in this report can be found [here](https://drive.google.com/file/d/1edlG-tOLPxLEWP19dBmBp_NxRmc7GAcx/view?usp=sharing). It contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Below is a schema diagram for the Chinook database:

![alt text](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)


### Importing the database

In the steps below `chinook.db` will be loaded in this Google Collaboratory notebook. 


In [1]:
# Import functions from Google modules into Colaboratory
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# Insert file id from Google Drive shareable link:
# https://drive.google.com/file/d/1edlG-tOLPxLEWP19dBmBp_NxRmc7GAcx/view?usp=sharing
id = '1edlG-tOLPxLEWP19dBmBp_NxRmc7GAcx'

In [3]:
# Download the dataset
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('chinook.db')

In [4]:
# Import SQLite3 and pandas library
import sqlite3
import pandas as pd

In [5]:
# Connect the notebook to the database file
%%capture
%load_ext sql
%sql sqlite:///chinook.db

To verify the database is fully functional, a query is run to return information on the tables and views in the database.

In order to run SQL queries in a Collab notebook, the IPython Magic Command `%%sql` needs to be run on the first line of every cell, followed by the SQL query itself. 

In [6]:
%%sql

SELECT name, type
  FROM sqlite_master
 WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 2. Best selling product categories

In this section, the following questions will be answered:
* Who are the best selling artists?
* What are the best selling tracks?
* What are the best selling albums? 
* What are the best selling genres?
* Which artists are most sold through playlists?

### Best selling artists

In [7]:
%%sql

SELECT
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10;

 * sqlite:///chinook.db
Done.


artist,tracks_purchased
Queen,192
Jimi Hendrix,187
Nirvana,130
Red Hot Chili Peppers,130
Pearl Jam,129
AC/DC,124
Guns N' Roses,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


**Queen** and **Jimi Hendrix** are most popular artists in the Chinook music store.

Using the query above in reversed order, the least popular artists can be found that could be eliminated from the database to cut costs for Chinook. Chinooks management would have to decide before proceeding with this, so currently it remains outside the scope of this project. A date clause would have to be included to avoid elimination of artists that have recently been added to the inventory.

### Best selling tracks

In [10]:
%%sql

SELECT
    t.name track,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10;

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such column: t.name
[SQL: SELECT
    t.name track,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


### Best selling albums

In [9]:
%%sql

SELECT
    ta.album_title album,
    ta.artist_name artist,
    COUNT(*) tracks_purchased
  FROM invoice_line il
 INNER JOIN (
            SELECT
                t.track_id,
                al.title album_title,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
 GROUP BY 1, 2
 ORDER BY 3 DESC 
 LIMIT 5;

 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


The album "**Are You Experienced?**" by **Jimi Hendrix** is sold most according to the database. Below is an overview of the tracks on this album.

In [12]:
%%sql

SELECT * FROM
    (
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )
WHERE album_name = "Are You Experienced?";

 * sqlite:///chinook.db
Done.


name,artist,album_name,media_type,genre,length_milliseconds
Foxy Lady,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,199340
Manic Depression,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,222302
Red House,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,224130
Can You See Me,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,153077
Love Or Confusion,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,193123
I Don't Live Today,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,235311
May This Be Love,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,191216
Fire,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,164989
Third Stone From The Sun,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,404453
Remember,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,168150


![Jimi Hendrix](https://1.bp.blogspot.com/-_KAun5P_CU0/UI9S8RSQ7eI/AAAAAAAAAhc/K2XrzWc2_rY/s1600/Jimi+Hendrix.jpg)

### Best selling genres

In [11]:
%%sql 

SELECT 
       g.name genre_name,
       SUM(il.quantity) tracks_sold FROM track t
 INNER JOIN invoice_line il ON il.track_id = t.track_id INNER JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre_name,tracks_sold
Rock,2635
Metal,619
Alternative & Punk,492
Latin,167
R&B/Soul,159
Blues,124
Jazz,121
Alternative,117
Easy Listening,74
Pop,63


**Rock** is by far the best selling genre on Chinook, followed by **Metal** and **Alternative & Punk**. It is advisable for Chinook to leverage its strength in these genres, and invest to expand its inventory in these genres. In its marketing expressions, Chinook should profile itself as a **specialist** in these genres to further attrack audiences interested in these genres.

The opposite, more **generalist** approach of boosting sales efforts in weaker genres (such as Soundtracks, Hip Hop/Rap and Reggae) could expand the user base substantially, but might be less cost efficient. 

[**The Long Tail: Why the Future of Business is Selling Less of More**](https://www.amazon.com/Long-Tail-Future-Business-Selling/dp/1401309666) by Chris Anderson describes why this generalist might pay off in the long term, but further marketing research would have to be done.

## 3. Sales performance per country

This section will answer the following question: **What countries generate most sales, and which the least?**

In [16]:
%%sql
WITH cust_country AS
                (
                    SELECT c.customer_id,
                            c.country ,
                            COUNT(i.invoice_id) no_of_purchases,
                            SUM(i.total) total_purchase_per_cust
                    FROM customer c 
                    INNER JOIN invoice i ON i.customer_id = c.customer_id
                    GROUP BY 1,2
                    ORDER BY 2
                ),
    country_group AS
                (
                    SELECT cc.country,
                           COUNT(cc.customer_id) total_customers,
                           SUM(cc.total_purchase_per_cust) Total_sales,
                            SUM(cc.total_purchase_per_cust)/SUM(cc.no_of_purchases) AVG_order_value,
        
                            CASE 
                                WHEN COUNT(cc.customer_id) = 1 THEN 'Other'
                                ELSE cc.country   
                            END AS country1
        
           
             
                    FROM cust_country cc
                    GROUP BY 1
                ),
        country_sort AS
                    (
                        SELECT 
                            cg.country1 country,
                            SUM(cg.total_customers) Total_customers,
                            SUM(cg.Total_sales) Total_sales,
                            SUM(cg.Total_sales)/SUM(cg.Total_customers) AVG_Sales_Per_Customer,
                            cg.AVG_order_value Avg_order_value,
                            CASE
                                WHEN cg.country1='Other' THEN 1
                                ELSE 0
                            END AS sort 
                        FROM country_group cg
                        GROUP BY 1
                        ORDER BY sort ASC ,Total_sales DESC
                    )
SELECT cs.country Country,
       cs.Total_customers Total_customers,
       ROUND(cs.Total_sales,2) Total_sales,
    ROUND(cs.AVG_sales_Per_Customer,2) AVG_sales_Per_Customer,
       ROUND(cs. Avg_order_value,2) AVG_order_value
FROM country_sort cs

 * sqlite:///chinook.db
Done.


Country,Total_customers,Total_sales,AVG_sales_Per_Customer,AVG_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.56,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.52


The **USA** generates the most sales, followed by **Canada**, while average sales per customer are the lowest for these countries. **Czech Republic** has the highest average sale per customer and the highest average order value. It would be advisable to concentrate marketing efforts in these countries. The countries with the lowest sales are **Portugal** and **India**. 

## 4. Sales agents performance

## 5. Customer base

## 6. Music inventory